In [1]:
import argparse
import RetrieveImage as retrieveImage
import os
import csv
import string
import re
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from datetime import *
import json
import pandas as pd
import metadata_preprocessing as processing
import xgboost as xgb
import DeepImageUtils as IU
from sklearn.linear_model import Ridge
from lightgbm import LGBMRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import csr_matrix, hstack,find
import keras

from keras import optimizers, callbacks
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import Model
from keras.layers import Input, Dropout, Dense
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

2021-10-14 07:01:55.160094: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
data_path = os.path.abspath(globals()['_dh'][0] + '/..') + '/Data'
# images_path = "/ceph/csedu-scratch/project/akaradathodi/Images/"
images_path = os.path.abspath(data_path + '/../../') + '/Images/'

image_features_path = os.path.abspath(globals()['_dh'][0] + '/..') + '/Data/total_features.npz'

In [3]:
def run_on_xgb_regressor(train_data, y_train, test_data):
    xgb_model = xgb.XGBRegressor()
    xgb_model.fit(train_data, y_train)
    predictions = xgb_model.predict(test_data)
    return predictions

In [4]:
def run_on_ridge_regression(train_data, y_train, test_data):
    ridge_model = Ridge(solver='lsqr', fit_intercept=False)  # solver='lsqr' reduces time to train significantly
    ridge_model.fit(train_data, y_train)

    predictions = ridge_model.predict(test_data)
    return predictions

In [5]:
def run_on_light_bgm_regressor(train_data, y_train, test_data):
    lgbm_model = LGBMRegressor()
    lgbm_model.fit(train_data, y_train)
    predictions = lgbm_model.predict(test_data)
    return predictions

In [21]:
def find_rmse_mae_r2(predicted_price_xgb, predicted_price_ridge, predicted_price_lightbgm, actual_price):
    predicted_price_xgb = np.array(predicted_price_xgb)
    predicted_price_ridge = np.array(predicted_price_ridge)
    predicted_price_lightbgm = np.array(predicted_price_lightbgm)
    actual_price = np.array(actual_price)

    results = {
            'MSE_xgb': mean_squared_error(predicted_price_xgb, actual_price),
            'MAE_xgb': mean_absolute_error(predicted_price_xgb, actual_price),
            'R2_xgb': r2_score(predicted_price_xgb, actual_price),
            'MSE_ridge': mean_squared_error(predicted_price_ridge, actual_price),
            'MAE_ridge': mean_absolute_error(predicted_price_ridge, actual_price),
            'R2_ridge': r2_score(predicted_price_ridge, actual_price),
            'MSE_lightbgm': mean_squared_error(predicted_price_lightbgm, actual_price),
            'MAE_lightbgm': mean_absolute_error(predicted_price_lightbgm, actual_price),
            'R2_lightbgm': r2_score(predicted_price_lightbgm, actual_price)
        }
    with open(data_path + '/output/feature_stacking_clothing_' + datetime.now().strftime(
                "%Y-%m-%d %H:%M:%S") + '.json', 'w') as file:
            json.dump(results, file)

In [6]:
def find_rmse_mae_r2_for_one_method(predicted_price, actual_price):
    predicted_price = np.array(predicted_price)
    actual_price = np.array(actual_price)

    results = {
            'MSE': mean_squared_error(predicted_price, actual_price),
            'MAE': mean_absolute_error(predicted_price, actual_price),
            'R2': r2_score(predicted_price, actual_price),
        }
    return results

In [7]:
def get_predicted_prices_by_feature_vectors(test_df, validation_df, train_df):
    # train_df, validation_df = add_features_to_df(train_df, validation_df)
    # test_df = add_features_to_testdf(test_df)
    X_test = test_df.drop(['Unnamed: 0', 'care_instructions','images', 'image_0', 'image_1', 'image_2', 'price'],
                     axis=1)
    y_test = test_df['price']

    y_train = train_df['price']
    X_train = train_df.drop(['Unnamed: 0','images','care_instructions','image_0','image_1','image_2','price'], axis = 1)

    y_validation = validation_df['price']
    X_validation = validation_df.drop(
        ['Unnamed: 0', 'images', 'care_instructions', 'image_0', 'image_1', 'image_2', 'price'], axis=1)

    train_data, validation_data, test_data = generate_encodings_for_features(X_train, X_validation, X_test)


    print('--Final Data Matrix--')
    print(train_data.shape, y_test.shape)
        # print(test_data.shape, row['price'].shape)
        #do price prediction and return the price

    return train_data, validation_data, test_data, y_train, y_validation, y_test

In [8]:
def give_emoji_free_text(text):
    # return emoji.get_emoji_regexp().sub(r'', text)
    pattern = re.compile("["
                         u"\U0001F600-\U0001F64F"
                         u"\U0001F300-\U0001F5FF"
                         u"\U0001F680-\U0001F6FF"
                         u"\U0001F1E0-\U0001F1FF"
                         u"\U00002702-\U000027B0"
                         u"\U000024C2-\U0001F251"
                         "]+", flags=re.UNICODE)

    return pattern.sub(r'', text)

In [9]:
def remove_punchuations(sentence):
    # return text.translate(str.maketrans('', '', string.punctuation)).strip()
    regular_punct = list(string.punctuation)

    for punc in regular_punct:
        if punc in sentence:
            sentence = sentence.replace(punc, ' ')

    return sentence.strip()

In [10]:
def remove_contract_words(text):
    try:
        text = re.sub(r"won't", "will not", text)
        text = re.sub(r"can\'t", "can not", text)
        text = re.sub(r"n\'t", " not", text)
        text = re.sub(r"\'re", " are", text)
        text = re.sub(r"\'s", " is", text)
        text = re.sub(r"\'d", " would", text)
        text = re.sub(r"\'ll", " will", text)
        text = re.sub(r"\'t", " not", text)
        text = re.sub(r"\'ve", " have", text)
        text = re.sub(r"\'m", " am", text)
    except Exception as exp:
        print(exp)
    return text

In [11]:
STOPWORDS = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"]

def remove_stopwords(text):
    # return ' '.join([word for word in text if word not in stopwords.words('english')])
    return ' '.join(e for e in text.split() if e not in STOPWORDS)

In [12]:
def process_text(data, cols):
    # print(data.columns.values.tolist())
    for col in cols:

        processed_data = []

        for sentence in data[col].values:
            sent = remove_contract_words(sentence)
            # sent = sentence
            try:
                sent = give_emoji_free_text(sent)
                sent = remove_punchuations(sent)
                sent = remove_stopwords(sent)
                sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
                sent = re.sub("\s+", " ", sent)
                sent = sent.lower().strip()
            except Exception as exp:
                print(exp)
            # if col == 'title':
            #     print(sent)
            processed_data.append(sent)

        data[col] = processed_data

    return data

In [13]:
def get_vohe(X_train, X_validation, X_test, col_name):
    """
    Get one hot encoded features
    """
    vect = CountVectorizer()
    tr_ohe = vect.fit_transform(X_train[col_name].values)
    vl_ohe = vect.transform(X_validation[col_name].values)
    te_ohe = vect.transform(X_test[col_name].values)

    return tr_ohe, vl_ohe, te_ohe

In [14]:
# tfidf word embeddings
def get_vtext_encodings(X_train, X_validation, X_test, col_name, min_val, max_val):
    """
    Get TFIDF encodings with max_features capped at 1M
    """
    vect = TfidfVectorizer(min_df=10, ngram_range=(min_val, max_val), max_features=1000000)
    tr_text = vect.fit_transform(X_train[col_name].values)
    vl_text = vect.transform(X_validation[col_name].values)
    te_text = vect.transform(X_test[col_name].values)

    return tr_text, vl_text, te_text

In [15]:
def generate_encodings_for_features(X_train, X_validation, X_test):
    """
        Get encodings for all the features. Scale and normalize the numerical features. Stack the encoded features horizontally.
    """
    X_train = X_train.fillna('others')
    X_test = X_test.fillna('others')
    X_validation = X_validation.fillna('others')
    X_train = process_text(X_train,
                           ['title', 'actual_color', 'product_details', 'complete_the_look', 'specifications', 'care_instruction_0', 'care_instruction_1'])

    X_validation = process_text(X_validation,
                                ['title', 'actual_color', 'product_details', 'complete_the_look', 'specifications', 'care_instruction_0', 'care_instruction_1'])

    X_test = process_text(X_test,
                          ['title', 'actual_color', 'product_details', 'complete_the_look', 'specifications', 'care_instruction_0', 'care_instruction_1'])

    tr_ohe_brand, vl_ohe_brand, te_ohe_brand = get_vohe(X_train, X_validation, X_test, 'brand')
    # tr_ohe_dominant_material, te_ohe_dominant_material = get_vohe(X_train, X_test, 'dominant_material')
    tr_ohe_dominant_color, vl_ohe_dominant_color, te_ohe_dominant_color = get_vohe(X_train, X_validation, X_test, 'dominant_color')
    tr_ohe_product_type, vl_ohe_product_type, te_ohe_product_type = get_vohe(X_train, X_validation, X_test, 'product_type')
    tr_ohe_gender, vl_ohe_gender, te_ohe_gender = get_vohe(X_train, X_validation, X_test, 'gender')
    tr_ohe_category_1, vl_ohe_category_1, te_ohe_category_1 = get_vohe(X_train, X_validation, X_test, 'category_1')
    tr_ohe_category_0, vl_ohe_category_0, te_ohe_category_0 = get_vohe(X_train, X_validation, X_test, 'category_0')

    tr_trans = csr_matrix(
        pd.get_dummies(X_train[['cheap_brands', 'expensive_brands', 'luxurious_brands']], sparse=True).values)
    vl_trans = csr_matrix(
        pd.get_dummies(X_validation[['cheap_brands', 'expensive_brands', 'luxurious_brands']], sparse=True).values)
    te_trans = csr_matrix(
        pd.get_dummies(X_test[['cheap_brands', 'expensive_brands', 'luxurious_brands']], sparse=True).values)

    tr_title, vl_title, te_title = get_vtext_encodings(X_train, X_validation, X_test, 'title', 1, 1)
    tr_product_details, vl_product_details, te_product_details = get_vtext_encodings(X_train, X_validation, X_test, 'product_details', 1, 2)
    tr_complete_the_look, vl_complete_the_look, te_complete_the_look = get_vtext_encodings(X_train, X_validation, X_test, 'complete_the_look', 1, 2)
    tr_specifications, vl_specifications, te_specifications = get_vtext_encodings(X_train, X_validation, X_test, 'specifications', 1, 2)
    tr_ohe_dominant_material, vl_ohe_dominant_material, te_ohe_dominant_material = get_vtext_encodings(X_train, X_validation, X_test, 'dominant_material', 1, 1)
    tr_ohe_actual_color, vl_ohe_actual_color, te_ohe_actual_color = get_vtext_encodings(X_train, X_validation, X_test, 'actual_color', 1, 1)
    tr_ohe_care_instruction_1, vl_ohe_care_instruction_1, te_ohe_care_instruction_1 = get_vtext_encodings(X_train, X_validation, X_test, 'care_instruction_1', 1, 1)
    tr_ohe_care_instruction_0, vl_ohe_care_instruction_0, te_ohe_care_instruction_0 = get_vtext_encodings(X_train, X_validation, X_test, 'care_instruction_0', 1, 1)


    train_data = hstack((tr_ohe_category_0, tr_ohe_category_1, tr_ohe_brand, tr_ohe_dominant_material,
                         tr_ohe_dominant_color, tr_ohe_actual_color, tr_ohe_product_type, tr_trans, tr_ohe_gender,
                         tr_ohe_care_instruction_0, tr_title, tr_product_details, tr_ohe_care_instruction_1,
                         tr_complete_the_look, tr_specifications)).tocsr().astype('float32')

    validation_data = hstack((vl_ohe_category_0, vl_ohe_category_1, vl_ohe_brand, vl_ohe_dominant_material,
                         vl_ohe_dominant_color, vl_ohe_actual_color, vl_ohe_product_type, vl_trans, vl_ohe_gender,
                         vl_ohe_care_instruction_0, vl_title, vl_product_details, vl_ohe_care_instruction_1,
                         vl_complete_the_look, vl_specifications)).tocsr().astype('float32')

    # validation_data = np.concatenate((validation_features_in_order, validation_data))

    test_data = hstack((te_ohe_category_0, te_ohe_category_1, te_ohe_brand, te_ohe_dominant_material,
                        te_ohe_dominant_color, te_ohe_actual_color, te_ohe_product_type, te_trans, te_ohe_gender,
                        te_ohe_care_instruction_0, te_title, te_product_details, te_ohe_care_instruction_1,
                        te_complete_the_look, te_specifications)).tocsr().astype('float32')
    # test_data = np.concatenate((test_features_in_order, validation_data))

    return train_data, validation_data, test_data

In [19]:
test_df = pd.read_csv(data_path + "/test_records_cloth.csv")
validation_df = pd.read_csv(data_path + "/validation_records_cloth.csv")
train_df = pd.read_csv(data_path + "/train_records_cloth.csv")

predicted_price_xgb = []
predicted_price_ridge = []
predicted_price_lightbgm = []
actual_price = []

train_df.iloc[0]

Unnamed: 0                                                            0
uniq_id                                7b5b9b1854e0ae2edcb99e7dd839f06a
product_id                                                      8350233
size                                                                XXL
variant_sku                                                 8350233_XXL
brand                                                           Sangria
care_instructions                                             Hand Wash
dominant_material                                                   NaN
title                 Sangria Women Red & Orange Printed Straight Fi...
actual_color                                               Red | Orange
dominant_color                                                      Red
product_type                                                        NaN
images                http://assets.myntassets.com/v1/assets/images/...
body                  Minimal yet stylish, this orange kurta fro

In [20]:
train_data, validation_data, test_data, y_train, y_validation, y_test = get_predicted_prices_by_feature_vectors(test_df, validation_df, train_df)

--Final Data Matrix--
(6332, 5724) (2108,)


In [22]:
predicted_price_xgb.append(run_on_xgb_regressor(train_data, y_train, test_data))

In [23]:
predicted_price_ridge.append(run_on_ridge_regression(train_data, y_train, test_data))

In [24]:
predicted_price_lightbgm.append(run_on_light_bgm_regressor(train_data, y_train, test_data))

In [25]:
find_rmse_mae_r2(predicted_price_xgb[0], predicted_price_ridge[0], predicted_price_lightbgm[0], y_test)

### Hyperparameter Tuning

In [26]:
#values of these parameters are taken randomly
xgb_params = {'gamma': 0.2, 'reg_lambda': 0.5, 'learning_rate': 0.07,
              'subsample': 0.7, 'colsample_bytree': 0.9, 'reg_alpha': 0.6,
              'max_depth': 6}

estimators = [50, 100, 300, 500, 1000]

for i in estimators:
    regr = xgb.XGBRegressor(n_estimators=i)
    regr.set_params(**xgb_params)

    regr.fit(train_data, y_train)
    y_pred = regr.predict(validation_data)
    
    print('Test rmsle explained for n_estimators = {} is {}'.format(i, str(find_rmse_mae_r2_for_one_method(y_pred, y_validation))))

Test rmsle explained for n_estimators = 50 is {'MSE': 760531.9439087531, 'MAE': 548.4593718673727, 'R2': 0.6181427874716697}
Test rmsle explained for n_estimators = 100 is {'MSE': 672767.9068108031, 'MAE': 505.4893763926571, 'R2': 0.7015190163114022}
Test rmsle explained for n_estimators = 300 is {'MSE': 612337.8294105135, 'MAE': 459.15013331826043, 'R2': 0.7489999101545507}
Test rmsle explained for n_estimators = 500 is {'MSE': 602669.811906893, 'MAE': 449.52933617471706, 'R2': 0.7578924954366755}
Test rmsle explained for n_estimators = 1000 is {'MSE': 607084.7323634732, 'MAE': 447.39324450348767, 'R2': 0.7598254359099315}


### XGB with best parameters

In [26]:
xgb_params = {'gamma': 0.2, 'reg_lambda': 0.5, 'learning_rate': 0.07,
              'subsample': 0.7, 'colsample_bytree': 0.9, 'reg_alpha': 0.6,
              'max_depth': 6}

xgb_model = xgb.XGBRegressor(n_estimators = 500)
xgb_model.set_params(**xgb_params)

xgb_model.fit(train_data, y_train)

predictions_xgb = xgb_model.predict(test_data)
result = find_rmse_mae_r2_for_one_method(predictions_xgb, y_test)

In [27]:
result

{'MSE': 492853.69426420436, 'MAE': 432.2104842531613, 'R2': 0.7766914993990468}

In [28]:
with open(data_path + '/output/feature_stacking_xgb_best_hyperparameters_Clothing_' + datetime.now().strftime(
                "%Y-%m-%d-%H:%M:%S") + '.json', 'w') as file:
            json.dump(result, file)